In [3]:
import numpy as np
import pandas as pd
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [27]:
# Helper to write pred
def write_to_submission_file(predicted_labels, out_file,
                            target="target", index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                               index = np.arange(1, predicted_labels.shape[0] + 1), 
                               columns =[target])
    predicted_df.to_csv(out_file, index_label = index_label)

In [5]:
train_df = pd.read_csv('train_sessions.csv',
                       index_col='session_id', parse_dates=['time1'])
test_df = pd.read_csv('test_sessions.csv',
                       index_col='session_id', parse_dates=['time1'])

# Sort by time
train_df = train_df.sort_values(by='time1')

train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [6]:
sites = ['site%s' % i for i in range (1,11)]
train_df[sites].fillna(0).astype('int').to_csv('train_sessions_text.txt',
                                             sep = ' ', index=None, header=None)
test_df[sites].fillna(0).astype('int').to_csv('test_sessions_text.txt',
                                             sep = ' ', index=None, header=None)


In [15]:
%%time
cv = CountVectorizer(ngram_range=(1,3), max_features=50000)
with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
    
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.fit_transform(inp_test_file)

X_train.shape,
X_test.shape

Wall time: 12 s


In [16]:
# Saving train targets into separated vector

y_train = train_df['target'].astype('int').values

In [17]:
# TimeSeries cross-validation from sklearn

time_split = TimeSeriesSplit(n_splits=10)

In [18]:
[(el[0].shape, el[1].shape) for el in time_split.split(X_train)]

[((23051,), (23051,)),
 ((46102,), (23051,)),
 ((69153,), (23051,)),
 ((92204,), (23051,)),
 ((115255,), (23051,)),
 ((138306,), (23051,)),
 ((161357,), (23051,)),
 ((184408,), (23051,)),
 ((207459,), (23051,)),
 ((230510,), (23051,))]

In [19]:
logit = LogisticRegression(C=1, random_state=42, solver='liblinear')

In [20]:
%%time
cv_scores = cross_val_score(logit, X_train, y_train, cv=time_split,
                           scoring='roc_auc', n_jobs=1)

Wall time: 1min 23s


In [24]:
cv_scores, cv_scores.mean()

(array([0.83141992, 0.64669953, 0.87992157, 0.9631551 , 0.84221458,
        0.87840646, 0.94475732, 0.85321751, 0.92987691, 0.90752901]),
 0.8677197894166472)

In [25]:
# Train logistic regression with all training data, make predictions for test set and form a submission file

logit.fit(X_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [28]:
logit_test_pred = logit.predict_proba(X_test)[:, 1]
write_to_submission_file(logit_test_pred, 'submission_file.csv')

In [34]:
def add_time_features(df, X_sparse):
    hour = df['time1'].apply(lambda ts: ts.hour)
    morning = ((hour>=6) & (hour<=11)).astype('int')
    afternoon = ((hour>=12) & (hour<=18)).astype('int')
    evening = ((hour>=19) & (hour<=23)).astype('int')
    night = ((hour>=0) & (hour<=5)).astype('int')
    
    X = hstack([X_sparse, morning.values.reshape(-1,1),
                afternoon.values.reshape(-1,1),
                evening.values.reshape(-1,1),
                night.values.reshape(-1,1)])
    return X

In [35]:
%%time
X_train_new = add_time_features(train_df.fillna(0), X_train)
X_test_new = add_time_features(test_df.fillna(0), X_test)

Wall time: 1.91 s


In [36]:
X_train_new.shape, X_test_new.shape

((253561, 50004), (82797, 50004))

In [38]:
%%time
cv_scores = cross_val_score(logit, X_train_new, y_train, cv=time_split,
                           scoring='roc_auc', n_jobs=1)

Wall time: 50.7 s


In [39]:
cv_scores, cv_scores.mean()

(array([0.87652264, 0.75123185, 0.93062022, 0.97864183, 0.90399748,
        0.93831379, 0.96249405, 0.92731398, 0.94886187, 0.94043454]),
 0.9158432245537969)

In [40]:
logit.fit(X_train_new, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [41]:
logit_test_pred2 = logit.predict_proba(X_test_new)[:, 1]
write_to_submission_file(logit_test_pred2, 'submission_file2.csv')

In [42]:
# tune C

In [44]:
c_values = np.logspace(-2, 2, 10)

logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=1, cv=time_split, verbose=1)

In [45]:
%%time
logit_grid_searcher.fit(X_train_new, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\DenisKOMAROV\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\DenisKOMAROV\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\DenisKOMAROV\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\DenisKOMAROV\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 12.8min finished


Wall time: 12min 52s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'C': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [46]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.9173784450124846, {'C': 0.21544346900318834})

In [47]:
logit_test_pred3 = logit_grid_searcher.predict_proba(X_test_new)[:, 1]
write_to_submission_file(logit_test_pred3, 'submission_file3.csv')